Homework 4: Sentiment Analysis - Task 2
----

Names 
----
Names: __Jalen Wu, Jonathan Zhang__ (Write these in every notebook you submit.)

Task 2: Train a Naive Bayes Model (30 points)
----

Using `nltk`'s `NaiveBayesClassifier` class, train a Naive Bayes classifier using a Bag of Words as features.

Learn more about Naive Bayes here: https://www.nltk.org/_modules/nltk/classify/naivebayes.html 

Naive Bayes classifiers use Bayes’ theorem for predictions. Naive Bayes can be a good baseline for NLP applications in particular. You can use it as a baseline for your project!

**

**10 points in Task 5 will be allocated for all 9 graphs (including the one generated here in Task 4 for Naive Bayes Classifier) being:**
- Legible
- Present below
- Properly labeled
     - x and y axes labeled
     - Legend for accuracy measures plotted
     - Plot Title with which model and run number the graph represents

In [2]:
# our utility functions
# RESTART your jupyter notebook kernel if you make changes to this file
import sentiment_utils as sutils

# nltk for Naive Bayes and metrics
import nltk
import nltk.classify.util
from nltk.metrics.scores import (precision, recall, f_measure, accuracy)
from nltk.classify import NaiveBayesClassifier

# some potentially helpful data structures from collections
from collections import defaultdict, Counter

# so that we can make plots
import matplotlib.pyplot as plt
# if you want to use seaborn to make plots
import seaborn as sns

[nltk_data] Downloading package punkt to /Users/jalenwu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# define constants for the files we are using
TRAIN_FILE = "movie_reviews_train.txt"
DEV_FILE = "movie_reviews_dev.txt"

In [4]:
# load in your data and make sure you understand the format
# Do not print out too much so as to impede readability of your notebook

# Generates a list of tuples from each file. Each tuple is a tokenized sentence.
train_tups = sutils.generate_tuples_from_file(TRAIN_FILE)
dev_tups = sutils.generate_tuples_from_file(DEV_FILE)

In [12]:
print((train_tups[1][1]))
# print(dev_tups[0][0])

0


In [27]:
# set up a sentiment classifier using NLTK's NaiveBayesClassifier and 
# a bag of words as features
# take a look at the function in lecture notebook 7 (feel free to copy + paste that function)
# the nltk classifier expects a dictionary of features as input where the key is the feature name
# and the value is the feature value

# TODO: Key: Feature Name, Value: Feature Value

# need to return a dict to work with the NLTK classifier
# Possible problem for students: evaluate the difference 
# between using binarized features and using counts (non binarized features)

# 0 = bad, 1 = good

# train_tups[0][0] is a sentence in the training set
# train_tups[1][0] is the sentiment of the sentence in the training set (0 bad 1 good)

# TODO: 1. what is the dictionary supposed to have as keys and values? keys are words and values are sentiment? But then what if the same word is in sentences that are both good and bad?
# 2. how to pass in the dictionary to the classifier? What would be arg1 and arg2?

# (__init__(label_probdist, feature_probdist)[source])) <- constructor for NaiveBayesClassifier

# list(tuple(dictionary{words : True}, label (0 or 1))) for each example in the training set

# TODO: Implement word_feats function
def word_feats(tuple_list) -> dict:    
    """
    This function converts a list of words so that they are featurized
    for nltk's format for bag-of-words.
    """
    return dict([(word, True) for word in tuple_list])


# set up & train a sentiment classifier using NLTK's NaiveBayesClassifier and
# classify the first example in the dev set as an example
# make sure your output is well-labeled

# list(tuple(dictionary{words : True}, label (0 or 1))) for each example in the training set

train_list = list()

for i in range(len(train_tups[0])):
    feats = word_feats(train_tups[0][i])
    sentiment = train_tups[1][i]
    temp = tuple((feats, sentiment))
    train_list.append(temp)

nbc = NaiveBayesClassifier.train(train_list)

# test to make sure that you can train the classifier and use it to classify a new example


In [28]:
print(dev_tups[0][0])
dev_tups_dict = word_feats(dev_tups[0][0])

output = nbc.classify(dev_tups_dict)
print(output)

['The', 'movie', "'Gung", 'Ho', '!', "'", ':', 'The', 'Story', 'of', 'Carlson', "'s", 'Makin', 'Island', 'Raiders', 'was', 'made', 'in', '1943', 'with', 'a', 'view', 'to', 'go', 'up', 'the', 'moral', 'of', 'American', 'people', 'at', 'the', 'duration', 'of', 'second', 'world', 'war', '.', 'It', 'shows', 'with', 'the', 'better', 'way', 'that', 'the', 'cinema', 'can', 'constitute', 'body', 'of', 'propaganda', '.', 'The', 'value', 'of', 'this', 'film', 'is', 'only', 'collection', 'and', 'no', 'artistic', '.', 'In', 'a', 'film', 'of', 'propaganda', 'it', 'is', 'useless', 'to', 'judge', 'direction', 'and', 'actors', '.', 'Watch', 'that', 'movie', 'if', 'you', 'are', 'interested', 'to', 'learn', 'how', 'propaganda', 'functions', 'in', 'the', 'movies', 'or', 'if', 'you', 'are', 'a', 'big', 'fun', 'of', 'Robert', 'Mitchum', 'who', 'has', 'a', 'small', 'role', 'in', 'the', 'film', '.', 'If', 'you', 'want', 'to', 'see', 'a', 'film', 'for', 'the', 'second', 'world', 'war', ',', 'they', 'exist', '

In [ ]:
# Using the provided dev set, evaluate your model with precision, recall, and f1 score as well as accuracy
# You may use nltk's implemented `precision`, `recall`, `f_measure`, and `accuracy` functions
# (make sure to look at the documentation for these functions!)
# you will be creating a similar graph for logistic regression and neural nets, so make sure
# you use functions wisely so that you do not have excessive repeated code
# write any helper functions you need in sentiment_utils.py (functions that you'll use in your other notebooks as well)


# create a graph of your classifier's performance on the dev set as a function of the amount of training data
# the x-axis should be the amount of training data (as a percentage of the total training data)
# NOTE : make sure one of your experiments uses 10% of the data, you will need this to answer the first question in task 5
# the y-axis should be the performance of the classifier on the dev set
# the graph should have 4 lines, one for each of precision, recall, f1, and accuracy
# the graph should have a legend, title, and axis labels



Test your model using both a __binarized__ (bag of words representation where we put 1 [true] if the word is there and 0 [false] otherwise) and a __multinomial__ (bag of words representation where we put the count of the word if the word occurs, and 0 otherwise). Use whichever one gives you a better final f1 score on the dev set to produce your graphs.

- f1 score binarized: __YOUR ANSWER HERE__
- f1 score multinomial: __YOUR ANSWER HERE__